# Introduction to Data Science Course Final Project

> by Kfir Goldfarb

<a href="https://github.com/kggold4"><img src="images/github.png" width="25px" height="25px" align="left"></a>
<a href="https://www.linkedin.com/in/kfir-goldfarb/"><img src="images/in.png"  width="25px" height="25px" align="left"></a>
<a href="mailto:kfir.goldfarb@msmail.ariel.ac.il"><img src="images/email.png" width="25px" height="25px" align="left"></a>

<br>

# Jupyter Notebook Number 3
<i>Machine Learning</i>
<br>

<img src="images/ml2images.jpg" width="90%" height="auto" align="left">